In [ ]:
# [Cell 1] 기본 라이브러리 및 설정
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 설정값
INPUT_FILE = "project_full_context.txt"
OUTPUT_JSON = "project_flows.json"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 실행 환경: {device.upper()}")

🚀 실행 환경: CUDA


In [ ]:
# [Cell 2] DeepSeek-R1-Distill-Qwen-1.5B 로딩 (초고속 추론)

# ★ 모델 변경: 2025년형 초소형 추론 모델 (1.5B)
MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

print(f"🚀 실행 환경: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")

# 1.5B는 너무 작아서 4비트 안 해도 되지만, 더 빠른 속도를 위해 적용
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

try:
    print(f"🔄 모델 로딩 중... ({MODEL_ID})")
    print("   -> 1.5B 모델이라 다운로드도 금방 끝납니다.")
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="eager"
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    print("✅ 모델 로딩 완료! (준비 끝)")

except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")

🚀 실행 환경: CUDA (GPU)
🔄 모델 로딩 중... (Qwen/Qwen2.5-Coder-1.5B-Instruct)
✅ 모델 로딩 완료!


In [4]:
# [Cell 3] 프롬프트 매니저 (계층형 트리 구조 전용)

# 수정된 SYSTEM_PROMPT (DB 및 미들웨어 추적 강화)

SYSTEM_PROMPT = """
You are a 'Backend Code Analyst'.
Your task is to generate a **Precise Call Graph** based ONLY on the provided code.

### CRITICAL RULES (DO NOT HALLUCINATE)
1. **Identify Database Driver**: Check `package.json` or imports. Is it `mongoose`, `mysql2`, or `pg`?
   - If `mongoose` is used, DO NOT output SQL queries like `SELECT *`. Use Mongoose methods like `find()`, `save()`.
2. **Trace Middleware**: If a router has `isAuth` or `validate`, you MUST include them in `children`.
3. **Deep Trace**: Go from Router -> Middleware -> Controller -> Service/Data -> DB Library.

### JSON OUTPUT FORMAT
{
  "api": [
    {
      "category": "auth",
      "categoryName": "Auth Feature",
      "endpoints": [
        {
          "method": "POST",
          "url": "/auth/signup",
          "function": "signup",
          "file": "controller/auth.mjs",
          "description": "User Signup",
          "children": [
            {
              "function": "validateSignup",
              "file": "router/auth.mjs",
              "description": "Validation Chain",
              "children": []
            },
            {
              "function": "bcrypt.hashSync",
              "file": "controller/auth.mjs",
              "description": "Password Hashing",
              "children": []
            },
            {
              "function": "createUser",
              "file": "data/auth.mjs",
              "description": "Save to DB",
              "children": [
                {
                   "function": "User.save()",
                   "file": "mongoose",
                   "description": "MongoDB Insert",
                   "children": []
                }
              ]
            }
          ]
        }
      ]
    }
  ]
}
"""

print("✅ 계층형 분석 프롬프트 설정 완료")

✅ 계층형 분석 프롬프트 설정 완료


In [ ]:
# [Cell 4] 코드 분석 및 JSON 생성 (R1 1.5B 전용)

def extract_json(text):
    # <think> 태그 제거 (생각 과정은 JSON에 넣지 않음)
    text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()
    text = re.sub(r"^```(json)?", "", text, flags=re.MULTILINE)
    text = re.sub(r"```$", "", text, flags=re.MULTILINE)
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1: return "{}"
    return text[start:end+1]

def run_full_scan():
    if not os.path.exists(INPUT_FILE):
        print(f"❌ '{INPUT_FILE}' 파일이 없습니다.")
        return

    print(f"📂 '{INPUT_FILE}' 읽는 중...")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        full_code = f.read()
        
    code_context = full_code[:30000] # 1.5B는 문맥을 너무 길게 주면 힘들어함

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Analyze this code structure deeply and return the JSON.\n\nCode:\n{code_context}"}
    ]

    text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    print("🧠 Tiny R1이 추론 중입니다... (속도: 🚀)")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2048,
        temperature=0.6,
        do_sample=True
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 결과 확인 (생각 과정이 궁금하면 아래 주석 해제)
    print(response) 
    
    json_str = extract_json(response)
    
    try:
        data = json.loads(json_str)
        with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✅ JSON 생성 성공! -> {OUTPUT_JSON}")
        return data
    except json.JSONDecodeError as e:
        print("❌ JSON 파싱 실패. AI 응답을 확인하세요.")
        print(response[:500])
        return None

# 실행
json_data = run_full_scan()

📂 'project_full_context.txt' 읽는 중...
✅ 완벽한 JSON 파일이 생성되었습니다! -> project_flows.json


In [ ]:
# [Cell 5] 텍스트 트리 뷰어 (JSON 구조 확인용)

def print_tree(node, prefix="", is_last=True):
    """재귀적으로 JSON 트리를 출력"""
    connector = "└── " if is_last else "├── "
    
    # 노드 이름 결정 (function 또는 method+url)
    if "method" in node:
        name = f"[{node['method']}] {node['url']} ({node.get('description', '')})"
    elif "function" in node:
        name = f"ƒ {node['function']} - {node.get('description', '')}"
    elif "category" in node:
        name = f"📂 Category: {node.get('categoryName', node['category'])}"
    else:
        name = "Unknown Node"

    print(prefix + connector + name)
    
    # 자식 노드 순회
    children = node.get("children", [])
    if "endpoints" in node:
        children = node["endpoints"]
        
    count = len(children)
    for i, child in enumerate(children):
        new_prefix = prefix + ("    " if is_last else "│   ")
        print_tree(child, new_prefix, i == count - 1)

def visualize_json_structure():
    if not os.path.exists(OUTPUT_JSON):
        print("❌ JSON 파일이 없습니다.")
        return

    with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print("\n🌳 API Call Graph Analysis Result\n" + "="*40)
    if "api" in data:
        for cat in data["api"]:
            print_tree(cat)
    else:
        print("⚠️ 'api' 키를 찾을 수 없습니다. JSON 구조를 확인하세요.")

# 실행
visualize_json_structure()


🌳 API Call Graph Analysis Result
└── 📂 Category: Auth Feature
    ├── [POST] /auth/signup (User Signup)
    │   ├── ƒ validateSignup - Validation Chain
    │   ├── ƒ bcrypt.hashSync - Password Hashing
    │   └── ƒ createUser - Save to DB
    │       └── ƒ User.save() - MongoDB Insert
    ├── [POST] /auth/login (User Login)
    │   ├── ƒ validateLogin - Validation Chain
    │   ├── ƒ bcrypt.compare - Password Verification
    │   ├── ƒ createJwtToken - Create JWT Token
    │   └── ƒ res.status(200).json({ token, userid }) - Send JWT Token and User ID
    └── [GET] /auth/me (Get Current User Info)
        ├── ƒ authRepository.findById(req.id) - Find User by ID
        └── ƒ res.status(200).json({ token, userid }) - Send JWT Token and User ID
